# Lab 4: Budget and Timeout Controls

## Objectives
- Add timeout controls to ping script from Lab 3
- Implement budget cap functionality
- Create cost tracking and budget monitoring
- **Exit ticket**: Paste run showing budget exceeded flag

In [ ]:
# Setup and imports
!pip install asksageclient pip_system_certs
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import time
import tiktoken
from pathlib import Path
from typing import Dict, List, Any

# Import our AskSage client
from asksageclient import AskSageClient

# Get API credentials from Google Colab secrets
from google.colab import userdata
api_key = userdata.get('ASKSAGE_API_KEY')
email = userdata.get('ASKSAGE_EMAIL')

# Initialize client and tokenizer
client = AskSageClient(api_key=api_key, email=email)
tokenizer = tiktoken.encoding_for_model("gpt-4")
print("AskSage client initialized successfully")
print("Ready to showcase AI capabilities...")

## Task 1: Budget Tracker (15 minutes)

In [ ]:

# Single Request

# GOAL:
# - Build the full prompt (system + user), count INPUT tokens
# - Send AskSage request
# - Extract reply, count OUTPUT tokens
# - Compute EXACT cost with input+output
# - Update budget (deny if cannot afford)
#
# HINTS:
# - Use: client.query(message=..., system_prompt=..., temperature=0.0, model="gpt-5-mini", live=0, limit_references=0)
# - Use: extract_text(...) to get reply string

def call_once_with_budget(
    user_prompt: str,
    system_prompt: str = "You are a helpful assistant.",
    model: str = "gpt-5-mini",
    budget: Dict | None = None,
) -> Dict:
    # 1) TODO: Build full_prompt and count INPUT tokens
    # full_prompt = ...
    # input_tokens = ...
    full_prompt = f"System Prompt: {system_prompt}\nMessage: {user_prompt}"
    input_tokens = count_tokens(full_prompt)

    # 2) OPTIONAL: pre-check with input-only estimate (useful UI)
    _, est_in_cost = budget_estimate(model, system_prompt, user_prompt)
    if budget and not budget_can_afford(budget, est_in_cost):
        # NOTE: For grading, final decision uses exact cost after model reply.
        pass

    # 3) TODO: Create client and send the request
    client = get_asksage_client()
    raw = client.query(
        message=user_prompt,
        system_prompt=system_prompt,
        temperature=0.0,
        model=model,
        live=0,
        limit_references=0,
    )

    # 4) TODO: Extract reply + count OUTPUT tokens
    reply_text = extract_text(raw)
    output_tokens = count_tokens(reply_text)

    # 5) TODO: Compute EXACT total cost (input + output)
    total_cost = cost_usd(model, input_tokens, output_tokens)

    # 6) TODO: Apply budget logic (deny or add expense)
    budget_exceeded = False
    if budget:
        if not budget_can_afford(budget, total_cost):
            budget_exceeded = True
        else:
            snap = budget_add_expense(budget, total_cost)
            budget_exceeded = snap["budget_exceeded"]

    # 7) Return small result dict
    return {
        "success": True,
        "model": model,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "cost_usd": total_cost,
        "budget_exceeded": budget_exceeded,
        "total_spent": (budget["spent"] if budget else 0.0),
        "reply": reply_text,
    }

# ===== Student Run (edit the prompt and budget as desired) =====
# demo_budget = budget_init(0.01)
# result = call_once_with_budget("Say hello in exactly five words.", budget=demo_budget)
# console.print(result)


In [ ]:
#@ Loop Until Budget Exceeded

# GOAL:
# - Start with a very small budget (e.g., $0.002)
# - Iterate through short prompts (1 sentence each)
# - For each:
#   - Call 'call_once_with_budget'
#   - Display tokens, cost, cumulative spend
#   - Stop as soon as budget_exceeded is True
#
# TODOs:
#  - Choose a tiny budget so this triggers quickly
#  - Provide 3–5 short prompts
#  - Print a neat table row (model, in/out tokens, cost, spent, exceeded)

def run_until_exceeded(
    prompts: List[str],
    system_prompt: str = "Respond in one short sentence.",
    model: str = "gpt-5-mini",
    limit_usd: float = 0.002,  # TODO: adjust to trigger budget exceeded
):
    budget = budget_init(limit_usd)
    table = Table(title=f"Lab 4 — Budget Run (limit=${limit_usd:.4f})")
    table.add_column("#", justify="right")
    table.add_column("Prompt", overflow="fold", max_width=40)
    table.add_column("Input toks", justify="right")
    table.add_column("Output toks", justify="right")
    table.add_column("Cost (USD)", justify="right")
    table.add_column("Spent (USD)", justify="right")
    table.add_column("Exceeded", justify="center")

    for i, p in enumerate(prompts, 1):
        r = call_once_with_budget(
            user_prompt=p,
            system_prompt=system_prompt,
            model=model,
            budget=budget,
        )
        table.add_row(
            str(i),
            p,
            str(r["input_tokens"]),
            str(r["output_tokens"]),
            f'{r["cost_usd"]:.6f}',
            f'{r["total_spent"]:.6f}',
            "🚨" if r["budget_exceeded"] else "",
        )
        if r["budget_exceeded"]:
            console.print(table)
            console.print("[bold red]Budget exceeded — stop here.[/bold red]")
            return r

    console.print(table)
    console.print("[bold green]Completed without exceeding budget.[/bold green]")
    return {"success": True, "budget": budget}

# ===== Student Run (fill in your prompts and un-comment) =====
# my_prompts = [
#     "Explain tokens in one short sentence.",
#     "Give a different one-line definition of tokens.",
#     "Another short explanation about token counting.",
#     "One more short sentence about costs."
# ]
# _ = run_until_exceeded(my_prompts, limit_usd=0.002)


In [ ]:
#Log Results to JSONL 

# GOAL:
# - For each call, append a JSONL line with the key fields needed for later analysis/dashboards.
# - Fields to log per row (suggested):
#   - timestamp, model, system_prompt, prompt, reply
#   - input_tokens, output_tokens, cost_usd
#   - total_spent, budget_exceeded

from datetime import datetime
import jsonlines

def log_result_jsonl(row: Dict, path: str = "./lab4_runs.jsonl"):
    """Append a single result dict to a JSON Lines file."""
    with jsonlines.open(path, mode="a") as writer:
        writer.write(row)

def run_and_log(
    prompts: List[str],
    system_prompt: str = "One short sentence.",
    model: str = "gpt-5-mini",
    limit_usd: float = 0.003,
    jsonl_path: str = "./lab4_runs.jsonl",
):
    budget = budget_init(limit_usd)
    for p in prompts:
        r = call_once_with_budget(
            user_prompt=p,
            system_prompt=system_prompt,
            model=model,
            budget=budget,
        )
        row = {
            "timestamp": datetime.now().isoformat(timespec="seconds"),
            "model": model,
            "system_prompt": system_prompt,
            "prompt": p,
            "reply": r["reply"],
            "input_tokens": r["input_tokens"],
            "output_tokens": r["output_tokens"],
            "cost_usd": r["cost_usd"],
            "total_spent": r["total_spent"],
            "budget_exceeded": r["budget_exceeded"],
        }
        # TODO: Append any extra fields your team wants to analyze later
        log_result_jsonl(row, jsonl_path)
        if r["budget_exceeded"]:
            console.print(f"[bold red]Stopped early: budget exceeded at ${r['total_spent']:.6f}[/bold red]")
            break

# ===== Student Run (uncomment to generate a JSONL you can inspect) =====
# prompts_for_log = [
#     "Define token in <= 12 words.",
#     "Define embedding in <= 12 words.",
#     "Define context window in <= 12 words."
# ]
# run_and_log(prompts_for_log, limit_usd=0.003, jsonl_path="./lab4_runs.jsonl")
# console.print("Logged to ./lab4_runs.jsonl")


## 📝 Exit Ticket

Run the budget test and paste the output showing "BUDGET EXCEEDED" flag.

The output should show:
```
🚨 [provider]: BUDGET EXCEEDED! ($X.XXXX spent)
```

## 🎯 What You've Built
- Budget tracking and enforcement
- Timeout controls for API calls
- Cost estimation system
- Production-ready safety controls

## 🚀 Next Steps
Day 2 will cover advanced prompt engineering techniques!